# Elevation Accuracy

This notebook creates a pipeline which determines both the most accurate interpolation method for a series of elevation points within Minnesota and the accuracy of the point data through comparison of known elevation data. The data is then stored in a file geodatabase and moved to an .sde database for creating real-time web maps in ArcGIS Online.

## Add and organize elevation data from SDE database

In [1]:
import os
import psycopg2
from psycopg2 import sql
import arcpy
import random
import requests
import zipfile
import io
import pandas as pd
import arcgis

In [2]:
# Establish variables for file path
file_path = os.path.dirname(arcpy.mp.ArcGISProject('CURRENT').filePath)
os.chdir(file_path)

# Estabish workspace and spatial reference as WGS 84
arcpy.env.workspace = file_path
spatial_ref = arcpy.SpatialReference(4326)

# Establish variables for project and map
project = arcpy.mp.ArcGISProject("CURRENT")
m = project.listMaps("Map")[0]

In [4]:
# Create a file GDB to add data to
arcpy.management.CreateFileGDB(
    out_folder_path = file_path,
    out_name = 'Lab_3.gdb'
)

<Result 'C:\\Users\\15612\\Documents\\Arc II\\Lab 3\\Lab 3\\Lab_3.gdb'>

In [3]:
# Establish variables for the file and SDE GDB path
file_gdb = os.path.join(file_path, 'Lab_3.gdb')
sde_gdb = os.path.join(file_path, 'PostgreSQL-35-gis5572(postgres).sde')

## Connect to SDE database

In [5]:
# Create database connection
dbname = 'PostgreSQL-35-gis5572(postgres).sde'
platform = 'POSTGRESQL'
user = 'postgres'
password = 'Hyderabad43%'
instance = '35.238.64.215'
port = '5432'
auth = 'DATABASE_AUTH'
save = 'SAVE_USERNAME'
db = 'gis5572'

arcpy.management.CreateDatabaseConnection(
    out_folder_path = file_path,
    out_name = dbname,
    database_platform = platform,
    instance = instance,
    account_authentication = auth,
    username = user,
    password = password,
    save_user_pass = save,
    database = db
)

<Result 'C:\\Users\\15612\\Documents\\Arc II\\Lab 3\\Lab 3\\PostgreSQL-35-gis5572(postgres).sde'>

## Add "ground truth" DEM

In [6]:
# Define a function to pull data from the Minnesota Geospatial Commons
def mn_geo_pull_and_unzip(url, directory, GDB_or_SHP):

    # Get GeoJSON from MN Geospatial Commons
    api = requests.get(url)
    json = api.json()
    
    # Use second list in the 'resources' key if data is in file geodatabase format
    if GDB_or_SHP == 'GDB':
        zip_link = requests.get(json['result']['resources'][1]['url'])
    
    # Use first list in the 'resources' key if data is in shapefile format
    if GDB_or_SHP == 'SHP':
        zip_link = requests.get(json['result']['resources'][2]['url'])
    
    # Get zipfile and extract
    z_file = zipfile.ZipFile(io.BytesIO(zip_link.content))
    z_file.extractall(directory)

In [7]:
mn_geo_pull_and_unzip('https://gisdata.mn.gov/api/3/action/package_show?id=elev-30m-digital-elevation-model',r'C:\Users\15612\Documents\Arc II\Lab 3\Lab 3\DEM','GDB')

In [8]:
m.addDataFromPath(r'C:\Users\15612\Documents\Arc II\Lab 3\Lab 3\DEM\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m')

In [24]:
# Reproject to perform resampling and converting to points
arcpy.management.ProjectRaster(
    in_raster = r'C:\Users\15612\Documents\Arc II\Lab 3\Lab 3\DEM\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m',
    out_raster = os.path.join(file_gdb,'digital_elevation_model_30m'),
    out_coor_system = spatial_ref
)

# Resample to limit the number of output points
arcpy.management.Resample(
    in_raster = os.path.join(file_gdb,'digital_elevation_model_30m'),
    out_raster = os.path.join(file_gdb,'digital_elevation_model_5km'),
    cell_size = 0.139 # About 5 km in vertical degrees
)

# Establish extent as the size of the ground truth DEM
arcpy.env.extent = os.path.join(file_gdb,'digital_elevation_model_5km')

In [6]:
print("Extracting data from SDE database and adding XY data...")

# Define the SDE feature class path
sde_feature_class = "gis5572.postgres.mn_dem_pts"

# Define the local feature class path
local_feature_class = r"C:\Users\15612\Documents\Arc II\Lab 3\Lab 3\Lab_3.gdb\MN_DEM_pts"

# Process: Copy Features
arcpy.CopyFeatures_management(r"C:\Users\15612\Documents\Arc II\Lab 3\Lab 3\PostgreSQL-35-gis5572(postgres).sde\gis5572.postgres.mn_dem_pts", local_feature_class)

# Add x-coordinate field
arcpy.management.AddField(
    in_table = os.path.join(file_gdb,'MN_DEM_pts'),
    field_name = 'X',
    field_type = 'DOUBLE'
)

# Calculate x-coordinate
arcpy.management.CalculateGeometryAttributes(
    in_features = os.path.join(file_gdb,'MN_DEM_pts'),
    geometry_property = [['X','POINT_X']],
    coordinate_system = spatial_ref
)
  
# Add y-coordinate field
arcpy.management.AddField(
    in_table = os.path.join(file_gdb,'MN_DEM_pts'),
    field_name = 'Y',
    field_type = 'DOUBLE'
)

# Calculate y-coordinate
arcpy.management.CalculateGeometryAttributes(
    in_features = os.path.join(file_gdb,'MN_DEM_pts'),
    geometry_property = [['Y','POINT_Y']],
    coordinate_system = spatial_ref
)

print("Done")

Extracting data from SDE database and adding XY data...
Done


## 50/50 Sampling

The function below creates a random sample of a specified percentage of points the user inputs for further exploratory interpolation within the notebook. See below for documentation:

### create_sample(input_feature_class {str}, x_field {str}, y_field {str}, z_field {str}, percent {float}, gdb_path {str}, out_feature_class {str})
- input_feature_class - feature class where the randomly sampled points will be extracted from 
- id_field - field with ID of each feature
- x_field - field with x coordinate
- y_field - field with y coordinate
- z_field - field with value to be interpolated 
- percent - ratio of points (in decimal notation) taken from original point feature class 
- gdb_path - path of geodatabase to save the created feature class to
- out_feature_class - name of the output feature class

In [10]:
def create_elev_sample(input_feature_class, id_field, x_field, y_field, z_field, percent, gdb_path, out_feature_class):
    
    print('Creating feature class "' + out_feature_class + '"...')
    
    # Create an empty list to add to
    all_rows = []
    
    # Append the XYZ data to the empty list
    with arcpy.da.SearchCursor(in_table = input_feature_class, field_names = [id_field, z_field, x_field, y_field]) as cursor:
        for row in cursor:
            name = row[0]
            Z = row[1]
            X = row[2]
            Y = row[3]
            all_rows.append([name, Z, X, Y])

    # Establish variables for the number of total rows, the number of samples, and the number of rows outside of the sample rows
    total_rows = len(all_rows)
    sample_num = int(total_rows * percent)
    removed_num = total_rows - sample_num

    # Randomize the order of all the rows
    random.shuffle(all_rows)
    
    # Create empty list for XYZ data
    name_list = []
    Z_list = []
    X_list = []
    Y_list = []

    # Add XYZ data to separate lists
    for row in all_rows:
        name_list.append(row[0])
        Z_list.append(row[1])
        X_list.append(row[2])
        Y_list.append(row[3])

    # Remove all of the XYZ data which is not part of the sampled rows
    df_list = [name_list, Z_list, X_list, Y_list]
    for l in df_list:
        del l[-removed_num:]

    # Create a dictionary with all XYZ data
    rand_dict = {
        'ID':name_list,
        'Z':Z_list,
        'X':X_list,
        'Y':Y_list
    }

    # Create a pandas dataframe from the dictionary
    random_sample_df = pd.DataFrame(rand_dict)
    
    # Convert dataframe to sedf
    sedf = arcgis.GeoAccessor.from_xy(
        df = random_sample_df, 
        x_column = "X",
        y_column = "Y"
    )

    # Convert sedf to feature class
    sedf.spatial.to_featureclass(location=os.path.join(gdb_path, out_feature_class))
    
    # Define projection for the created feature class
    arcpy.management.DefineProjection(
        in_dataset = out_feature_class,
        coor_system = spatial_ref
    )
    
    # Change field type for Z values to 'Float'
    arcpy.management.AddField(
        in_table = out_feature_class,
        field_name = z_field,
        field_type = 'FLOAT'
    )
    
    arcpy.management.CalculateField(
        in_table = out_feature_class,
        field = z_field,
        expression = '!z!'
    )
    
    arcpy.management.DeleteField(
        in_table = out_feature_class,
        drop_field = 'z'
    )
        
    print('Done')

In [13]:
# Create a feature class with 20% of random points sampled
create_elev_sample(
    input_feature_class = 'MN_DEM_pts',
    id_field = 'pointid',
    x_field = 'X',
    y_field = 'Y',
    z_field = 'grid_code',
    percent = 0.5,
    gdb_path = file_gdb,
    out_feature_class = 'Random_Sample_50_elevation'
)

Creating feature class "Random_Sample_50_elevation"...
Done


## Leave-One-Out Cross Validation

In [74]:
# Perform exploratory interpolation on three different interpolation techniques using random sample
arcpy.ga.ExploratoryInterpolation(
    in_features = 'Random_Sample_50_elevation',
    value_field = 'elevation',
    out_cv_table = os.path.join(file_gdb,'stats_table_elevation'),
    out_geostat_layer = None,
    interp_methods = ['ORDINARY_KRIGING','UNIVERSAL_KRIGING','IDW'],
    comparison_method = 'SINGLE',
    criterion = 'ACCURACY',
    criteria_hierarchy = 'ACCURACY PERCENT #',
    weighted_criteria = 'ACCURACY 1',
    exclusion_criteria = None
)

<Result 'C:\\Users\\15612\\Documents\\Arc II\\Lab 3\\Lab 3\\Lab_3.gdb\\stats_table'>

## Interpolations

In [15]:
# Ordinary kriging
elev_ord_krige = arcpy.sa.Kriging(
    in_point_features = 'Random_Sample_50_elevation',
    z_field = 'grid_code',
    kriging_model = arcpy.sa.KrigingModelOrdinary('SPHERICAL'),
    cell_size = 0.139
)

# Universal kriging
elev_uni_krige = arcpy.sa.Kriging(
    in_point_features = 'Random_Sample_50_elevation',
    z_field = 'grid_code',
    kriging_model = arcpy.sa.KrigingModelUniversal('LINEARDRIFT'),
    cell_size = 0.139
)

# Inverse distance weighting
elev_IDW = arcpy.sa.Idw(
    in_point_features = 'Random_Sample_50_elevation',
    z_field = 'grid_code',
    cell_size = 0.139
)

## Perform accuracy assessment
The function below creates points from the sampled and actual rasters created via the interpolations above and compares their Z-values (in this case, daily maximum temperature). The difference between temperature for each point is added to the field named "temp_difference" in the output accuracy feature class. The average difference between each temperature is calculated and output as a simple "print" statement with the lowest value representing the interpolation method with the highest accuracy.

### accuracy_analysis(sampled_interp_layer {str}, complete_interp_layer {str}, output_fc {str})
- sampled_interp_layer - the raster layer created via interpolation of the sampled points
- complete_interp_layer - the raster layer with the "ground truth" data
- output_fc - the name of the output feature class with the accuracy data

In [25]:
# Define a function to perform accuracy analysis and add accuracy data to feature class
def accuracy_analysis(sampled_interp_layer,complete_interp_layer,output_fc):

    print('Performing accuracy analysis for "' + sampled_interp_layer + '" and "' + complete_interp_layer + '"...')
    
    # Create a list of each interpolation layer
    interp_layer_list = [sampled_interp_layer,complete_interp_layer]
    
    # Iterate through each interpolation layer
    for layer in interp_layer_list:
    
        # Turn interpolation data to points
        arcpy.conversion.RasterToPoint(
            in_raster = layer,
            out_point_features = os.path.join(file_gdb,layer + '_points')
        )
        
    # Spatially join all points
    arcpy.analysis.SpatialJoin(
        target_features = os.path.join(file_gdb,sampled_interp_layer + '_points'),
        join_features = os.path.join(file_gdb,complete_interp_layer + '_points'),
        out_feature_class = os.path.join(file_gdb,output_fc),
        match_option = 'CLOSEST'
    )
    
    # Add a field to add the difference values to
    arcpy.management.AddField(
        in_table = os.path.join(file_gdb,output_fc),
        field_name = 'elev_difference',
        field_type = 'FLOAT'
    )
    
    # Calculate the difference between the actual and sampled weather station data
    try:
    
        arcpy.management.CalculateField(
            in_table = os.path.join(file_gdb,output_fc),
            field = 'elev_difference',
            expression = 'abs(!grid_code! - !grid_cod_1!)' 
        )
        
        arcpy.management.AlterField(
            in_table = os.path.join(file_gdb,output_fc),
            field = 'grid_cod_1',
            new_field_name = 'actual_elev',
            new_field_alias = 'actual_elev'
        )
        
    except:
        
        arcpy.management.CalculateField(
            in_table = os.path.join(file_gdb,output_fc),
            field = 'elev_difference',
            expression = 'abs(!grid_code! - !grid_code_1!)' # Autocreates table with name as "grid_cod_1" or "grid_code_1"
        )
        
        arcpy.management.AlterField(
            in_table = os.path.join(file_gdb,output_fc),
            field = 'grid_code_1',
            new_field_name = 'actual_elev',
            new_field_alias = 'actual_elev'
        )
    
    # Change names of fields to represent data
    arcpy.management.AlterField(
        in_table = os.path.join(file_gdb,output_fc),
        field = 'grid_code',
        new_field_name = 'sampled_elev',
        new_field_alias = 'sampled_elev'
    )
    
    # Delete excess fields
    arcpy.management.DeleteField(
        in_table = os.path.join(file_gdb,output_fc),
        drop_field = ['Join_Count','TARGET_FID','pointid_1']
    )
    
    # Delete sampled interpolation raster
    arcpy.management.Delete(
        in_data = os.path.join(file_gdb,sampled_interp_layer + '_points')
    )
    
    # Create empty list to append temperature difference values
    elev_dif_list = []
    
    # Iterate through each row and collect values, then calculate average to find average difference of all values
    with arcpy.da.SearchCursor(in_table = os.path.join(file_gdb,output_fc), field_names = 'elev_difference') as cursor:
        for row in cursor:
            elev_dif_list.append(row[0])
            
    print('The average elevation difference between "' + sampled_interp_layer + '" and "' + complete_interp_layer + '" is ' + str(sum(elev_dif_list)/len(elev_dif_list)))
    
    print('Done')

In [30]:
accuracy_analysis('elev_ord_krige','digital_elevation_model_5km','elev_ord_accuracy')

Performing accuracy analysis for "elev_ord_krige" and "digital_elevation_model_5km"...
The average elevation difference between "elev_ord_krige" and "digital_elevation_model_5km" is 90.43442914224265
Done


In [31]:
accuracy_analysis('elev_uni_krige','digital_elevation_model_5km','elev_uni_accuracy')

Performing accuracy analysis for "elev_uni_krige" and "digital_elevation_model_5km"...
The average elevation difference between "elev_uni_krige" and "digital_elevation_model_5km" is 375.20293396319823
Done


In [32]:
accuracy_analysis('elev_IDW','digital_elevation_model_5km','elev_idw_accuracy')

Performing accuracy analysis for "elev_IDW" and "digital_elevation_model_5km"...
The average elevation difference between "elev_IDW" and "digital_elevation_model_5km" is 110.67556910863073
Done


## Results

Ordinary kriging appears to have the smallest mean difference between point values, meaning it is likely the most accurate when observing elevation values. 

The points created via IDW interpolation will be added to the SDE database in order to avoid confusion regarding which interpolation was most accurate.

In [33]:
points_list = ['elev_ord_accuracy','digital_elevation_model_5km_points']

for points in points_list:

    arcpy.conversion.FeatureClassToGeodatabase(
        Input_Features = points,
        Output_Geodatabase = sde_gdb
    )